In [0]:
%python
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ass").getOrCreate()

In [0]:
%python
df1=spark.read.csv("dbfs:/FileStore/tables/bank_fd_account-4.csv",inferSchema=True,header=True)
df2=spark.read.csv("dbfs:/FileStore/tables/bank_customer-4.csv",inferSchema=True,header=True)
df3=spark.read.csv("dbfs:/FileStore/tables/bank_rd_account-4.csv",inferSchema=True,header=True)
df4=spark.read.csv("dbfs:/FileStore/tables/bank_sb_account-4.csv",inferSchema=True,header=True)




In [0]:
%python
df1.display()


ACCOUNT_NO,CUST_ID,START_DT,FD_TERM_MNTH,INITIAL_AMT,ACC_STATUS
5212340001,1002,22-JUN-00,30,15000,Closed
5212340002,1002,11-MAY-02,66,25000,Active
5212340003,1007,26-NOV-03,36,80555,Closed
5212340004,1003,29-NOV-04,60,30000,Active
5212340005,1009,23-JUN-04,120,45000,Active
5212340006,1001,26-SEP-05,24,45000,Active
5212340007,1010,18-OCT-05,84,15000000,Active
5212340008,1008,18-DEC-06,36,98765,Active
5212340009,1006,14-FEB-07,120,25000,Active
5212340010,1007,21-SEP-08,60,2500000,Active


In [0]:
%python

df1.createOrReplaceTempView("bank_fd")
df2.createOrReplaceTempView("bank_customer")
df3.createOrReplaceTempView("bank_rd")
df4.createOrReplaceTempView("bank_sb")




In [0]:
%sql
select * from bank_customer


CUST_ID,CUST_FNAME,INITIALS,CUST_LNAME,CUST_SEX,CUST_DOB,CUST_TYPE
1001,John,null,Smith,M,21-OCT-78,IND
1002,Franklin,J,Dang,M,21-OCT-78,IND
1003,Alice,M,Powell,F,09-APR-76,IND
1004,Ramesh,R,Narayan,M,08-DEC-71,IND
1005,Nancy,S,Taylor,F,17-AUG-79,IND
1006,Ahmad,M A,Jabbar,M,25-JUN-72,IND
1007,DealWell Co.,null,null,null,null,CORP
1008,Aman,K,Mathur,M,30-OCT-80,IND
1009,Smitha,null,Ramesh,F,03-MAR-76,IND
1010,Popular Enterprises,null,null,null,null,CORP


In [0]:
%sql
select fd.cust_id,cust.cust_fname,fd.ACCOUNT_NO,fd.INITIAL_AMT from bank_fd as fd
join bank_customer as cust on fd.cust_id=cust.cust_id
where acc_status="Active" and INITIAL_AMT>25000
order by INITIAL_AMT desc

cust_id,cust_fname,ACCOUNT_NO,INITIAL_AMT
1010,Popular Enterprises,5212340007,15000000
1007,DealWell Co.,5212340010,2500000
1008,Aman,5212340008,98765
1009,Smitha,5212340005,45000
1001,John,5212340006,45000
1003,Alice,5212340004,30000


In [0]:
%sql
select * from bank_fd


ACCOUNT_NO,CUST_ID,START_DT,FD_TERM_MNTH,INITIAL_AMT,ACC_STATUS
5212340001,1002,22-JUN-00,30,15000,Closed
5212340002,1002,11-MAY-02,66,25000,Active
5212340003,1007,26-NOV-03,36,80555,Closed
5212340004,1003,29-NOV-04,60,30000,Active
5212340005,1009,23-JUN-04,120,45000,Active
5212340006,1001,26-SEP-05,24,45000,Active
5212340007,1010,18-OCT-05,84,15000000,Active
5212340008,1008,18-DEC-06,36,98765,Active
5212340009,1006,14-FEB-07,120,25000,Active
5212340010,1007,21-SEP-08,60,2500000,Active


In [0]:
%sql
select * from bank_rd

ACCOUNT_NO,RD_CUST_ID,RD_START_DT,RD_TERM_YRS,INSTALLMENT_AMT,CURR_BALANCE,ACC_STATUS
6000010001,1006,13-JUN-06,10,1000,75000,Active
6000010002,1003,26-NOV-06,6,3000,208000,Active
6000010003,1008,03-OCT-07,5,2500,148000,Active
6000010004,1009,29-APR-07,4,6000,0,Closed
6000010005,1001,17-FEB-08,1,10000,0,Closed
6000010006,1007,16-JUL-08,9,20000,1000012,Active
6000010007,1001,26-MAY-08,4,5000,0,Closed
6000010008,1010,18-DEC-09,7,50000,1550000,Active
6000010009,1007,20-APR-09,4,14000,560000,Active
6000010010,1010,08-MAY-10,7,25000,700000,Active


In [0]:
%sql
select * from bank_sb

ACCOUNT_NO,PRIMARY_CUST_ID,SECONDARY_CUST_ID,CURR_BAL_AMT,ACC_STATUS,START_DATE,END_DATE
1000012003,1005,1001,0.0,Closed,20-NOV-06,23-AUG-11
1000012004,1008,null,84924.0,Active,11-MAR-06,null
1000012005,1009,1004,209844.03,Active,20-NOV-06,null
1000012006,1004,null,0.0,Closed,12-SEP-07,12-SEP-10
1000012007,1003,null,12300.0,Inactive,09-DEC-08,null
1000012008,1005,null,10235.0,Active,23-MAY-09,null
1000012009,1006,null,73535.0,Active,10-OCT-10,null
1000012010,1004,null,54674.0,Active,09-OCT-11,null
1000012001,1001,null,25000.0,Active,06-SEP-04,null
1000012002,1002,null,31109.43,Active,16-DEC-05,null


In [0]:
%sql
select * from bank_rd as rd
join bank_sb as sb on rd.RD_CUST_ID=sb.PRIMARY_CUST_ID
join bank_fd as fd on rd.RD_CUST_ID=fd.CUST_ID
join bank_customer as cs on rd.RD_CUST_ID=cs.cust_id
where rd.ACC_STATUS='Active' and fd.ACC_STATUS='Active' and sb.ACC_STATUS='Active'

ACCOUNT_NO,RD_CUST_ID,RD_START_DT,RD_TERM_YRS,INSTALLMENT_AMT,CURR_BALANCE,ACC_STATUS,ACCOUNT_NO,PRIMARY_CUST_ID,SECONDARY_CUST_ID,CURR_BAL_AMT,ACC_STATUS,START_DATE,END_DATE,ACCOUNT_NO,CUST_ID,START_DT,FD_TERM_MNTH,INITIAL_AMT,ACC_STATUS,CUST_ID,CUST_FNAME,INITIALS,CUST_LNAME,CUST_SEX,CUST_DOB,CUST_TYPE
6000010001,1006,13-JUN-06,10,1000,75000,Active,1000012009,1006,null,73535.0,Active,10-OCT-10,null,5212340009,1006,14-FEB-07,120,25000,Active,1006,Ahmad,M A,Jabbar,M,25-JUN-72,IND
6000010003,1008,03-OCT-07,5,2500,148000,Active,1000012004,1008,null,84924.0,Active,11-MAR-06,null,5212340008,1008,18-DEC-06,36,98765,Active,1008,Aman,K,Mathur,M,30-OCT-80,IND


In [0]:
%sql
select cs.cust_id,cs.cust_fname,(rd.INSTALLMENT_AMT +fd.INITIAL_AMT+sb.CURR_BAL_AMT) as netw_worth from bank_rd as rd
join bank_sb as sb on rd.RD_CUST_ID=sb.PRIMARY_CUST_ID
join bank_fd as fd on rd.RD_CUST_ID=fd.CUST_ID
join bank_customer as cs on rd.RD_CUST_ID=cs.cust_id

where rd.ACC_STATUS='Active' and fd.ACC_STATUS='Active' and sb.ACC_STATUS='Active'

cust_id,cust_fname,netw_worth
1006,Ahmad,99535.0
1008,Aman,186189.0


CUST_ID,CUST_FNAME,INITIALS,CUST_LNAME,CUST_SEX,CUST_DOB,CUST_TYPE
1001,John,null,Smith,M,21-OCT-78,IND
1002,Franklin,J,Dang,M,21-OCT-78,IND
1003,Alice,M,Powell,F,09-APR-76,IND
1004,Ramesh,R,Narayan,M,08-DEC-71,IND
1005,Nancy,S,Taylor,F,17-AUG-79,IND
1006,Ahmad,M A,Jabbar,M,25-JUN-72,IND
1007,DealWell Co.,null,null,null,null,CORP
1008,Aman,K,Mathur,M,30-OCT-80,IND
1009,Smitha,null,Ramesh,F,03-MAR-76,IND
1010,Popular Enterprises,null,null,null,null,CORP


In [0]:
%python
df2.display()

CUST_ID,CUST_FNAME,INITIALS,CUST_LNAME,CUST_SEX,CUST_DOB,CUST_TYPE
1001,John,null,Smith,M,21-OCT-78,IND
1002,Franklin,J,Dang,M,21-OCT-78,IND
1003,Alice,M,Powell,F,09-APR-76,IND
1004,Ramesh,R,Narayan,M,08-DEC-71,IND
1005,Nancy,S,Taylor,F,17-AUG-79,IND
1006,Ahmad,M A,Jabbar,M,25-JUN-72,IND
1007,DealWell Co.,null,null,null,null,CORP
1008,Aman,K,Mathur,M,30-OCT-80,IND
1009,Smitha,null,Ramesh,F,03-MAR-76,IND
1010,Popular Enterprises,null,null,null,null,CORP


In [0]:
%python
from pyspark.sql.functions import dense_rank
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number


In [0]:
%python
df_new = df2.select("cust_type") \
           .distinct() \
           .withColumn("index", dense_rank().over(Window.orderBy("cust_type")))

In [0]:
%python
df_new.display()

cust_type,index
CORP,1
IND,2


In [0]:
%sql
select cs.cust_id,fd.ACCOUNT_NO as fd_ACCOUNT_NO ,fd.initial_amt,fd.acc_status as fd_status ,rd.ACCOUNT_NO as rd_ACCOUNT_NO,rd.INSTALLMENT_AMT as rd_amount,rd.acc_status as rd_status,sb.ACCOUNT_NO as sb_ACCOUNT_NO ,sb.CURR_BAL_AMT as current_balance ,sb.acc_status as sb_status from bank_customer as cs
left join bank_fd as fd on cs.cust_id=fd.cust_id
left join bank_rd as rd on cs.cust_id=rd.RD_CUST_ID
left join bank_sb as sb on cs.cust_id=sb.PRIMARY_CUST_ID
order by cs.cust_id

cust_id,fd_ACCOUNT_NO,initial_amt,fd_status,rd_ACCOUNT_NO,rd_amount,rd_status,sb_ACCOUNT_NO,current_balance,sb_status
1001,5212340006,45000,Active,6000010007,5000,Closed,1000012001,25000.0,Active
1001,5212340006,45000,Active,6000010005,10000,Closed,1000012001,25000.0,Active
1002,5212340002,25000,Active,null,null,null,1000012002,31109.43,Active
1002,5212340001,15000,Closed,null,null,null,1000012002,31109.43,Active
1003,5212340004,30000,Active,6000010002,3000,Active,1000012007,12300.0,Inactive
1004,null,null,null,null,null,null,1000012010,54674.0,Active
1004,null,null,null,null,null,null,1000012006,0.0,Closed
1005,null,null,null,null,null,null,1000012008,10235.0,Active
1005,null,null,null,null,null,null,1000012003,0.0,Closed
1006,5212340009,25000,Active,6000010001,1000,Active,1000012009,73535.0,Active


In [0]:
%sql
select cs.cust_id,fd.ACCOUNT_NO as fd_ACCOUNT_NO ,fd.initial_amt,fd.acc_status as fd_status ,rd.ACCOUNT_NO as rd_ACCOUNT_NO,rd.INSTALLMENT_AMT as rd_amount,rd.acc_status as rd_status,sb.ACCOUNT_NO as sb_ACCOUNT_NO ,sb.CURR_BAL_AMT as current_balance ,sb.acc_status as sb_status from bank_customer as cs
left join bank_fd as fd on cs.cust_id=fd.cust_id
left join bank_rd as rd on cs.cust_id=rd.RD_CUST_ID
left join bank_sb as sb on cs.cust_id=sb.PRIMARY_CUST_ID
case when fd_status='Active' then 
order by cs.cust_id 


ACCOUNT_NO,PRIMARY_CUST_ID,SECONDARY_CUST_ID,CURR_BAL_AMT,ACC_STATUS,START_DATE,END_DATE
1000012003,1005,1001,0.0,Closed,20-NOV-06,23-AUG-11
1000012004,1008,null,84924.0,Active,11-MAR-06,null
1000012005,1009,1004,209844.03,Active,20-NOV-06,null
1000012006,1004,null,0.0,Closed,12-SEP-07,12-SEP-10
1000012007,1003,null,12300.0,Inactive,09-DEC-08,null
1000012008,1005,null,10235.0,Active,23-MAY-09,null
1000012009,1006,null,73535.0,Active,10-OCT-10,null
1000012010,1004,null,54674.0,Active,09-OCT-11,null
1000012001,1001,null,25000.0,Active,06-SEP-04,null
1000012002,1002,null,31109.43,Active,16-DEC-05,null


In [0]:
%sql
SELECT bank_fd.CUST_ID, bank_fd.ACCOUNT_NO, bank_fd.INITIAL_AMT AS AMT, bank_fd.ACC_STATUS FROM bank_fd WHERE bank_fd.ACC_STATUS = 'Active'
UNION
SELECT bank_rd.RD_CUST_ID, bank_rd.ACCOUNT_NO, bank_rd.CURR_BALANCE, bank_rd.ACC_STATUS FROM bank_rd WHERE bank_rd.ACC_STATUS = 'Active'
UNION
SELECT bank_sb.PRIMARY_CUST_ID, bank_sb.ACCOUNT_NO, bank_sb.CURR_BAL_AMT, bank_sb.ACC_STATUS FROM bank_sb WHERE bank_sb.ACC_STATUS = 'Active'
ORDER BY CUST_ID, ACCOUNT_NO

CUST_ID,ACCOUNT_NO,AMT,ACC_STATUS
1001,1000012001,25000.0,Active
1001,5212340006,45000.0,Active
1002,1000012002,31109.43,Active
1002,5212340002,25000.0,Active
1003,5212340004,30000.0,Active
1003,6000010002,208000.0,Active
1004,1000012010,54674.0,Active
1005,1000012008,10235.0,Active
1006,1000012009,73535.0,Active
1006,5212340009,25000.0,Active


In [0]:
%sql
SELECT bank_fd.CUST_ID, bank_fd.ACCOUNT_NO, bank_fd.INITIAL_AMT AS AMT, bank_fd.ACC_STATUS FROM bank_fd WHERE bank_fd.ACC_STATUS = 'Active'
UNION
SELECT bank_rd.RD_CUST_ID, bank_rd.ACCOUNT_NO, bank_rd.CURR_BALANCE, bank_rd.ACC_STATUS FROM bank_rd WHERE bank_rd.ACC_STATUS = 'Active'
UNION
SELECT bank_sb.PRIMARY_CUST_ID, bank_sb.ACCOUNT_NO, bank_sb.CURR_BAL_AMT, bank_sb.ACC_STATUS FROM bank_sb WHERE bank_sb.ACC_STATUS = 'Active'
ORDER BY CUST_ID, ACCOUNT_NO